## Week 3 --> Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### 1. Transform data in the table on Wikipedia page into the given pandas dataframe

##### To build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

print("Installed successfully!")

Installed successfully!


In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import json
import xml
import pandas as pd

In [4]:
# To extract the html codes from wikipedia page
res = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df_Canada = BeautifulSoup(res.content,'lxml')

# To create a dataframe to store data from html code
col_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = col_names)

# Loop through html code to find postcode, borough, neighborhood

content = df_Canada.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0  
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')      
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

toronto.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [5]:
# To ignore cells with boroughs that are not assigned 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0

# To assign neighbourhood = borough if a cell has a borough but no assigned neighbourhood
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

# To group similar postalcodes with more than one neighborhood and separated them with a comma
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# To drop cells with boroughs that are not assigned 
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty) & (df.Borough != empty) & (df.Neighborhood != empty)]

# To group similar postalcodes with more than one neighborhood and separated them with a comma
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

df2.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# Shape of dataframe
print(df2.shape)

(103, 3)


### 2: Add the latitude and longitude coordinates to the dataframe

In [8]:
!conda install -c conda-forge geocoder -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py36_0   conda-forge
    ratelim:    0.1.6-py36_0 conda-forge


orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB     | ##################################### | 100% 
ratelim-0.1.6      

In [10]:
import geocoder

In [11]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

# Sample check on coordinates
get_latlng('M2H')

[43.80284500000005, -79.35623615099996]

In [12]:
# Retrieve postal codes
postal_codes = df['Postalcode']    
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [13]:
# Adding Latitude and Longitude to dataframe
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

df_coords.head()

,Latitude,Longitude
0,43.811650,-79.195561
1,43.785605,-79.158701
2,43.765690,-79.175299
3,43.768216,-79.217610
4,43.769608,-79.239440


In [26]:
# Sample check on data fields in new dataframe
df[df.Postalcode == 'M4G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude
38,M4G,East York,Leaside,43.709765,-79.363791


In [14]:
df.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069


## 3. Cluster the neighborhoods in Toronto

In [15]:
import requests
from bs4 import BeautifulSoup as bs
import re

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

In [16]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
etobicoke_data = df[df['Borough'] == 'Etobicoke'].reset_index(drop=True)
address1 = 'Etobicoke,Toronto'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude1, longitude1))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [24]:
map_etobicoke = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)
    
map_etobicoke

In [26]:
CLIENT_ID = '11EAVFNW2GBPYO2GQZPKXZAR1GZMVU2XMGIDLYPZ3IXIJCUK' # your Foursquare ID
CLIENT_SECRET = 'ZURQ1DXHFF4BX3CIXWIRFKZU3N345KRXMKV32ER2VUM1XGB3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)

Latitude and longitude values of Humber Bay Shores, Mimico South, New Toronto are 43.609920000000045, -79.49804936299995.


In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(12)

,name,categories,lat,lng
0,Tim Hortons,Coffee Shop,43.644705,-79.567659
1,Farmer's Market Etobicoke,Farmers Market,43.643061,-79.566191
2,Loblaws,Grocery Store,43.644287,-79.560023
3,State & Main Kitchen & Bar,American Restaurant,43.645750,-79.560231
4,TD Canada Trust,Bank,43.645692,-79.560180
5,GoodLife Fitness Etobicoke East Mall Women's Only,Gym,43.644267,-79.560149
6,West Mall Rink,Skating Rink,43.642138,-79.566218
7,Quiznos,Sandwich Place,43.642068,-79.565450
8,Rabba,Convenience Store,43.647096,-79.563026
9,Delimark Cafe,Café,43.647116,-79.562990


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Humber Bay Shores, Mimico South, New Toronto
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Northwest


In [38]:
etobicoke_venues.head(3)
print(etobicoke_venues.groupby('Neighborhood').count()[:4])

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Albion Gardens, Beaumond Heights, Humbergate, J...                     13   
Alderwood, Long Branch                                                  5   
Bloordale Gardens, Eringate, Markland Wood, Old...                      6   
Cloverdale, Islington, Martin Grove, Princess G...                      7   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Albion Gardens, Beaumond Heights, Humbergate, J...                      13   
Alderwood, Long Branch                                                   5   
Bloordale Gardens, Eringate, Markland Wood, Old...                       6   
Cloverdale, Islington, Martin Grove, Princess G...                       7   

                                                    Venue  Venue Lat

In [39]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()

In [41]:
# To extract top 10 venues per neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

In [45]:
etobicoke_grouped.head()

,Neighborhood,Arts & Crafts Store,Bank,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Carpet Store,Chinese Restaurant,Coffee Shop,College Rec Center,Convenience Store,Dance Studio,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Fried Chicken Joint,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Intersection,Italian Restaurant,Japanese Restaurant,Mattress Store,Middle Eastern Restaurant,Miscellaneous Shop,Music Venue,Park,Pharmacy,Pizza Place,Pool,Pub,Sandwich Place,Shopping Mall,Skating Rink,Storage Facility,Sushi Restaurant,Tea Room,Thai Restaurant,Video Store,Wine Shop,Yoga Studio
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.076923,0.0,0.0,0.0,0.0,0.000000,0.000000,0.076923,0.000000,0.000000,0.0,0.0,0.000000,0.076923,0.000000,0.076923,0.153846,0.0,0.0,0.076923,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.076923,0.076923,0.076923,0.0,0.0,0.076923,0.0,0.000000,0.0,0.0,0.000000,0.0,0.076923,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.200000,0.0,0.2,0.200000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.166667,0.000000,0.000000,0.166667,0.000000,0.0,0.0,0.166667,0.000000,0.166667,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,"Cloverdale, Islington, Martin Grove, Princess ...",0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.285714,0.0,0.0,0.142857,0.0,0.000000,0.0,0.0,0.142857,0.0,0.000000,0.142857,0.000000
4,"Humber Bay Shores, Mimico South, New Toronto",0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.166667,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.166667


In [47]:
# Using k-means to cluster into 5 clusters

# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 0, 3, 0, 1, 0, 4, 1, 3], dtype=int32)

In [50]:
etobicoke_merged = etobicoke_data

# add clustering labels
etobicoke_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etobicoke_merged.head(2)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.609920,-79.498049,0,Park,Yoga Studio,Grocery Store,Skating Rink,Convenience Store,College Rec Center,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
1,M8W,Etobicoke,"Alderwood, Long Branch",43.601196,-79.538885,3,Dance Studio,Sandwich Place,Pub,Gym,Pizza Place,Yoga Studio,College Rec Center,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant


In [53]:
# Cluster visualisation

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighborhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters